In [ ]:
import pandas as pd
import os
from IPython.display import FileLink

# --- CẤU HÌNH ---
base_path = "/kaggle/input/historical-hourly-weather-data/"

files_to_merge = {
    "temperature.csv": "temperature",
    "humidity.csv": "humidity",
    "pressure.csv": "pressure",
    "weather_description.csv": "weather_desc",
    "wind_direction.csv": "wind_direction",
    "wind_speed.csv": "wind_speed"
}

print("🚀 Bắt đầu xử lý và gộp dữ liệu cho TOÀN BỘ CÁC THÀNH PHỐ...")

processed_dfs = []

for filename, col_name in files_to_merge.items():
    file_path = os.path.join(base_path, filename)
    
    if os.path.exists(file_path):
        print(f"Processing {filename}...")
        df_temp = pd.read_csv(file_path)
        df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])
        
        # Melt: Chuyển cột thành phố thành dòng
        df_melted = df_temp.melt(id_vars=['datetime'], var_name='City', value_name=col_name)
        df_melted.set_index(['datetime', 'City'], inplace=True)
        processed_dfs.append(df_melted)
    else:
        print(f"❌ Không tìm thấy file {filename}")

# --- GỘP DỮ LIỆU ---
if processed_dfs:
    print("Đang gộp tất cả dữ liệu lại...")
    df_all = pd.concat(processed_dfs, axis=1)
    df_all.reset_index(inplace=True)

    print("Đang làm sạch dữ liệu...")

    # 1. Chuyển đổi Kelvin sang Celsius
    if 'temperature' in df_all.columns:
        df_all['temperature'] = pd.to_numeric(df_all['temperature'], errors='coerce')
        df_all['temperature'] = df_all['temperature'] - 273.15
        df_all['temperature'] = df_all['temperature'].round(2)

    # 2. Sắp xếp theo City trước để Fill Null chính xác
    # (Tránh lấy dữ liệu thành phố này đắp vào thành phố kia)
    df_all.sort_values(by=['City', 'datetime'], inplace=True)
    df_all = df_all.ffill().bfill()

    # --- 3. BƯỚC QUAN TRỌNG: SẮP XẾP LẠI THEO THỜI GIAN ---
    print("⏳ Đang sắp xếp dữ liệu theo thời gian (Datetime tăng dần)...")
    df_all.sort_values(by=['datetime', 'City'], ascending=[True, True], inplace=True)

    # --- 4. LƯU KẾT QUẢ (Lưu sau khi đã sort xong) ---
    output_filename = "all_cities_weather.csv"
    df_all.to_csv(output_filename, index=False)
    
    print("\n" + "="*30)
    print(f"✅ HOÀN TẤT! Đã tạo file: {output_filename}")
    print(f"Số dòng dữ liệu: {len(df_all)}")
    
    unique_cities = df_all['City'].unique()
    print(f"Số lượng thành phố: {len(unique_cities)}")
    print("="*30)
    
    print("👇 Tải file về tại đây:")
    display(FileLink(output_filename))
    
    print("\n--- Preview dữ liệu (Đã sort theo thời gian) ---")
    print(df_all.head(10)) # In 10 dòng đầu để kiểm tra

else:
    print("❌ Có lỗi xảy ra, không có dữ liệu nào được xử lý.")